In [1]:
from __future__ import unicode_literals
import pandas as pd 
import numpy as np
import os 

from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

from string import punctuation
from collections import Counter , defaultdict

from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

import gensim
import struct
from gensim.models.word2vec import Word2Vec